## Merge Labels and Texts

In [1]:
from utils.database import *
from utils.files import *
from tqdm import tqdm
from bson import ObjectId
import pandas as pd 
import numpy as np

## Load URLs

URLs per batch and topic.

In [2]:
df_labels = pd.read_json('../data/raw/pages_with_labels.json', orient='records', lines=True)
df_labels.head()

,p_id,url,used_at,duration,yt_video_id,package_version,enddevice,batch,Group,start_date,...,start_intervention,start_knowledge,topic,series,annotation_type,good_for_training,good_for_augmentation,category,is_direct_topic_annotated,is_direct_topic_full
0,273726366948,mingle.respondi.de/,2023-06-13 14:12:16,393,,1210041502.0,mobile,15,Search,2023-06-13 14:12:00,...,2023-06-13 14:18:35,2023-06-14 18:09:40,kinder,,domain_discarded,False,True,other,,False
1,273726366948,mingle.respondi.de/,2023-06-13 20:09:47,2,,1210041502.0,mobile,15,Search,2023-06-13 14:12:00,...,2023-06-13 14:18:35,2023-06-14 18:09:40,kinder,,domain_discarded,False,True,other,,False
2,273746614716,mingle.respondi.de/,2023-06-14 16:19:10,492,,1210041502.0,mobile,15,Control,2023-06-14 16:19:00,...,2023-06-14 16:26:33,2023-06-15 15:34:00,kinder,,domain_discarded,False,True,other,,False
3,273746614716,mingle.respondi.de/,2023-06-14 16:28:00,1,,1210041502.0,mobile,15,Control,2023-06-14 16:19:00,...,2023-06-14 16:26:33,2023-06-15 15:34:00,kinder,,domain_discarded,False,True,other,,False
4,273746614716,mingle.respondi.de/,2023-06-14 16:36:39,13,,1210041502.0,mobile,15,Control,2023-06-14 16:19:00,...,2023-06-14 16:26:33,2023-06-15 15:34:00,kinder,,domain_discarded,False,True,other,,False


In [3]:
print("Number of pages: ", len(df_labels))

Number of pages:  761232


In [4]:
df_labels.columns

Index(['p_id', 'url', 'used_at', 'duration', 'yt_video_id', 'package_version',
       'enddevice', 'batch', 'Group', 'start_date', 't', 'wave', 'end_date',
       'start_intervention', 'start_knowledge', 'topic', 'series',
       'annotation_type', 'good_for_training', 'good_for_augmentation',
       'category', 'is_direct_topic_annotated', 'is_direct_topic_full'],
      dtype='object')

In [5]:
def get_freqs(dataframe, value_column):
    """Prints the frequency distribution of a specified value column, grouped by 'p_id', as a DataFrame."""
    unique_df = dataframe.drop_duplicates(subset=['p_id', value_column])
    value_counts = unique_df[value_column].value_counts().reset_index()
    value_counts.columns = [value_column, 'Frequency']
    return value_counts

In [6]:
get_freqs(df_labels, 'category')

,category,Frequency
0,other,867
1,search,731
2,news,643


In [7]:
get_freqs(df_labels, 'good_for_training')

,good_for_training,Frequency
0,False,870
1,True,690


In [8]:
get_freqs(df_labels, 'annotation_type')

,annotation_type,Frequency
0,domain_discarded,868
1,03.searches,654
2,04.urls-with-title,647
3,09.other,637
4,13.searches,294
5,06.news-wo-title,246
6,01.youtube,209
7,07.sm,205
8,10.new_news,146
9,12.surfaced_domains,142


## Remove Duplicates

In [9]:
# Removing duplicates based on 'view_url', 'batch_id', and 'topic'
print("Number of pages with labels: ", len(df_labels))
df_labels = df_labels.drop_duplicates(subset=['url', 'batch', 'topic'], keep='first')
print("Number of pages with labels after removing duplicates: ", len(df_labels))

Number of pages with labels:  761232
Number of pages with labels after removing duplicates:  288979


## Load Texts

In [10]:
from datasets import load_dataset, Features, Value, Dataset

/home/jschelb/.pyenv/versions/3.10.8/envs/s2j-content-analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# File paths to load texts
file_path = f"../data/raw/pages_with_text.json"
save_path = f"../data/tmp/processed_dataset_all_topics_text_only"

In [12]:
# Define the schema 
features_schema = Features({
    '_id': Value('string'),
    'batch_id': Value('int64'),
    'domain': Value('string'),
    'encoding': Value('string'),
    'file_id': Value('string'),
    'lang': Value('string'),
    'text': Value('string'),
    'text_length': Value('int64'),
    'url': Value('string'),
    'word_count': Value('int64'),
    'view_url': Value('string'),
})

# Load the dataset from the JSON Lines file
dataset = load_dataset('json', data_files=file_path, split='train', features=features_schema, keep_in_memory=False)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1060.51it/s]
Generating train split: 3437316 examples [04:20, 13193.39 examples/s]


In [13]:
print(dataset)

Dataset({
    features: ['_id', 'batch_id', 'domain', 'encoding', 'file_id', 'lang', 'text', 'text_length', 'url', 'word_count', 'view_url'],
    num_rows: 3437316
})


In [14]:
# # Processing the dataset in batches and applying transformations
# def process_batch(batch):
#     return batch # TODO: Returning the batch as is to include all attributes from the JSON objects

# processed_dataset = dataset.map(process_batch, batched=True, batch_size=1000)

# Save the processed dataset to the specified folder
dataset.save_to_disk(save_path)
print(f"Dataset saved to {save_path}")

Saving the dataset (43/43 shards): 100%|██████████| 3437316/3437316 [00:19<00:00, 174772.71 examples/s]

Dataset saved to ../data/tmp/processed_dataset_all_topics_text_only


In [15]:
processed_dataset = dataset
processed_dataset

Dataset({
    features: ['_id', 'batch_id', 'domain', 'encoding', 'file_id', 'lang', 'text', 'text_length', 'url', 'word_count', 'view_url'],
    num_rows: 3437316
})

In [16]:
processed_dataset[0]

{'_id': '648c2ad88e8cadbd29004dff',
 'batch_id': 15,
 'domain': 'microsoft.com',
 'encoding': 'utf-8',
 'file_id': '648c67b41a16a1878d01c628',
 'lang': 'en',
 'text': " Learn more: Accessibility with Microsoft Edge Your browser has been updated to the newest version. Microsoft Edge is the fast and secure browser that helps you protect your data and save time. Next Browse with confidence using Browser essentials Stay informed about your browser's performance and security with a single intuitive view that provides helpful suggestions for performance optimization and browser protection. Go to Microsoft Edge's toolbar and select the Browser essentials heartbeat icon to get started. Learn more Next Get quick answers with Bing Chat Whether you're shopping online, planning your next trip, or comparing options -- simply ask a question at the side of your screen, and have Bing do the work for you. Try now Learn more Next Translate the web to your language Microsoft Edge makes it easy to read we

## Merge Labels and Text

In [17]:
# Convert the processed dataset to a pandas DataFrame
df_texts = processed_dataset.to_pandas()
df_labels.rename(columns={'url': 'view_url', 'batch': 'batch_id'}, inplace=True)
df_labels.head()

,p_id,view_url,used_at,duration,yt_video_id,package_version,enddevice,batch_id,Group,start_date,...,start_intervention,start_knowledge,topic,series,annotation_type,good_for_training,good_for_augmentation,category,is_direct_topic_annotated,is_direct_topic_full
0,273726366948,mingle.respondi.de/,2023-06-13 14:12:16,393,,1210041502.0,mobile,15,Search,2023-06-13 14:12:00,...,2023-06-13 14:18:35,2023-06-14 18:09:40,kinder,,domain_discarded,False,True,other,,False
7,273746614716,mingle.respondi.de/,2023-06-15 10:19:29,4,,1210041502.0,mobile,16,Control,2023-06-14 16:19:00,...,2023-06-14 16:26:33,2023-06-15 15:34:00,kinder,,domain_discarded,False,True,other,,False
504,273746614716,mingle.respondi.de/,2023-06-26 17:25:10,4,,1210306502.0,mobile,16,Control,2023-06-26 17:25:00,...,2023-06-26 17:34:21,2023-06-27 21:09:26,cannabis,,domain_discarded,False,True,other,,False
516,274671650160,mingle.respondi.de/,2023-07-01 22:26:38,4,,,desktop,17,Search+Financial,2023-07-01 22:31:00,...,2023-07-01 22:36:21,2023-07-02 21:11:12,cannabis,,domain_discarded,False,True,other,,False
949,273971922264,mingle.respondi.de/,2023-06-20 14:19:25,24,,,mobile,16,Control,2023-06-19 14:17:00,...,2023-06-19 14:25:29,2023-06-20 14:17:22,energie,,domain_discarded,False,True,other,,False


In [18]:
# Merge the text and labels DataFrames
df_texts_and_labels = pd.merge(df_texts, df_labels, 
                               left_on=["batch_id", "view_url"], 
                               right_on=["batch_id", "view_url"], 
                               how='right', suffixes=('_text', '_label'))

In [19]:
# Convert the 'good_for_training' column to boolean
df_texts_and_labels["is_topic"] = df_texts_and_labels["is_direct_topic_full"].map({'True': True, 'False': False})
df_texts_and_labels.columns

Index(['_id', 'batch_id', 'domain', 'encoding', 'file_id', 'lang', 'text',
       'text_length', 'url', 'word_count', 'view_url', 'p_id', 'used_at',
       'duration', 'yt_video_id', 'package_version', 'enddevice', 'Group',
       'start_date', 't', 'wave', 'end_date', 'start_intervention',
       'start_knowledge', 'topic', 'series', 'annotation_type',
       'good_for_training', 'good_for_augmentation', 'category',
       'is_direct_topic_annotated', 'is_direct_topic_full', 'is_topic'],
      dtype='object')

In [20]:
df_texts_and_labels[["batch_id", "domain", "lang", "text", "view_url", "topic", "is_topic"]].head(100)

,batch_id,domain,lang,text,view_url,topic,is_topic
0,15,NaN,NaN,NaN,mingle.respondi.de/,kinder,False
1,16,NaN,NaN,NaN,mingle.respondi.de/,kinder,False
2,16,NaN,NaN,NaN,mingle.respondi.de/,cannabis,False
3,17,NaN,NaN,NaN,mingle.respondi.de/,cannabis,False
4,16,NaN,NaN,NaN,mingle.respondi.de/,energie,False
...,...,...,...,...,...,...,...
95,16,aquafelix.it,it,x > CHI SIAMO > IL PARCO Le Attrazioni Mappa ...,aquafelix.it,energie,False
96,16,aquafelix.it,it,x > CHI SIAMO > IL PARCO Le Attrazioni Mappa ...,aquafelix.it/Acquisto/PrezziDinamici,energie,False
97,16,aquafelix.it,it,x > CHI SIAMO > IL PARCO Le Attrazioni Mappa ...,aquafelix.it/Home/ChiSiamo,energie,False
98,16,NaN,NaN,NaN,google.com/search?rlz=1C1CHBF_deDE871DE871&sxs...,energie,False


In [21]:
len(df_texts_and_labels)

288979

In [29]:
# Specify the column name
column_name = 'text'

# Count the number of NaN values in the specified column
num_nans_text = df_texts_and_labels[column_name].isna().sum()

# Count the number of non-NaN values in the specified column
num_non_nans_text = df_texts_and_labels[column_name].notna().sum()

# Alternatively, you can also calculate non-NaNs by subtracting NaNs from total elements in the column
total_elements_text = df_texts_and_labels[column_name].size
num_non_nans_text_alternative = total_elements_text - num_nans_text

print(f'Number of NaN values in column "{column_name}": {num_nans_text}')
print(f'Number of non-NaN values in column "{column_name}": {num_non_nans_text}')
print(f'Number of non-NaN values in column "{column_name}" (alternative calculation): {num_non_nans_text_alternative}')

Number of NaN values in column "text": 111761
Number of non-NaN values in column "text": 174487
Number of non-NaN values in column "text" (alternative calculation): 174487


## Remove Pages with same content across Batches

In [22]:
print("Number of pages with labels: ", len(df_texts_and_labels))

# Removing duplicates based on 'view_url', 'batch_id', and 'topic'
#df_texts_and_labels = df_texts_and_labels.drop_duplicates(subset=['view_url', 'text', 'topic'], keep='first')
df_texts_and_labels = df_texts_and_labels.drop_duplicates(subset=['view_url', 'topic'], keep='first') # Only keeping one text per page even if it changes

print("Number of pages with labels after removing duplicates: ", len(df_texts_and_labels))

Number of pages with labels:  288979
Number of pages with labels after removing duplicates:  286248


In [23]:
df_texts_and_labels[["batch_id", "domain", "lang", "text", "view_url", "topic", "is_topic"]].head(100)

,batch_id,domain,lang,text,view_url,topic,is_topic
0,15,NaN,NaN,NaN,mingle.respondi.de/,kinder,False
2,16,NaN,NaN,NaN,mingle.respondi.de/,cannabis,False
4,16,NaN,NaN,NaN,mingle.respondi.de/,energie,False
5,15,NaN,NaN,NaN,web.norstatsurveys.com/,kinder,False
6,16,NaN,NaN,NaN,web.norstatsurveys.com/,energie,False
...,...,...,...,...,...,...,...
101,16,NaN,NaN,NaN,norstatpanel.com/de/endsurvey?lang=de&status=s...,cannabis,False
103,16,NaN,NaN,NaN,norstatpanel.com/de/endsurvey?lang=de&status=s...,kinder,False
105,16,NaN,NaN,NaN,norstatpanel.com/de/endsurvey?lang=de&status=s...,energie,False
106,16,t-online.de,de,Wetter DAX Telefonverzeichnisse Lotto Telekom ...,email.t-online.de/em#f=INBOX&m=147888883217979...,cannabis,False


## Save Intermediate Datatset

In [24]:
# Convert the merged DataFrame back to a Dataset
dataset_texts_and_labels = Dataset.from_pandas(df_texts_and_labels)

In [25]:
save_path = f"../data/tmp/processed_dataset_all_topics"
dataset_texts_and_labels.save_to_disk(save_path)

Saving the dataset (3/3 shards): 100%|██████████| 286248/286248 [00:03<00:00, 73214.84 examples/s] 


In [26]:
dataset_texts_and_labels

Dataset({
    features: ['_id', 'batch_id', 'domain', 'encoding', 'file_id', 'lang', 'text', 'text_length', 'url', 'word_count', 'view_url', 'p_id', 'used_at', 'duration', 'yt_video_id', 'package_version', 'enddevice', 'Group', 'start_date', 't', 'wave', 'end_date', 'start_intervention', 'start_knowledge', 'topic', 'series', 'annotation_type', 'good_for_training', 'good_for_augmentation', 'category', 'is_direct_topic_annotated', 'is_direct_topic_full', 'is_topic', '__index_level_0__'],
    num_rows: 286248
})

## Split Dataset by Topic

In [27]:
topics = ["kinder", "energie", "cannabis"]

In [28]:
for topic in topics:
    
    print("Processing topic: ", topic)
    
    # Filter the dataset for the topic
    dataset_topic = dataset_texts_and_labels.filter(lambda example: example['topic'] == topic, num_proc=16)
    dataset_topic.save_to_disk(f"../data/tmp/processed_dataset_{topic}")
    print("Number of examples in dataset_topic: ", len(dataset_topic))
    
    # Filter the dataset for the topic and positive labels
    dataset_topic_positive = dataset_topic.filter(lambda example: example['is_topic'], num_proc=16)
    dataset_topic_positive.save_to_disk(f"../data/tmp/processed_dataset_{topic}_positive")
    print("Number of examples in dataset_topic_positive: ", len(dataset_topic_positive))
    
    # Filter the dataset for the topic and negative labels
    dataset_topic_negative = dataset_topic.filter(lambda example: not example['is_topic'], num_proc=16)
    dataset_topic_negative.save_to_disk(f"../data/tmp/processed_dataset_{topic}_negative")
    print("Number of examples in dataset_topic_negative: ", len(dataset_topic_negative))


Processing topic:  kinder


Saving the dataset (1/1 shards): 100%|██████████| 104397/104397 [00:05<00:00, 19481.27 examples/s]

Number of examples in dataset_topic:  104397



Saving the dataset (1/1 shards): 100%|██████████| 536/536 [00:00<00:00, 17951.14 examples/s]

Number of examples in dataset_topic_positive:  536


...

Number of examples in dataset_topic:  88236



Saving the dataset (1/1 shards): 100%|██████████| 309/309 [00:00<00:00, 9940.86 examples/s] 

Number of examples in dataset_topic_positive:  309



Saving the dataset (1/1 shards): 100%|██████████| 87927/87927 [00:04<00:00, 20637.34 examples/s]

Number of examples in dataset_topic_negative:  87927
